In [15]:
%load_ext autoreload
%autoreload 2

from gnn_tracking.utils.loading import TrackingDataModule
from pytorch_lightning import Trainer
from gnn_tracking.training.tc import TCModule
from pathlib import Path
from rich.pretty import pprint
from pytorch_lightning.utilities.model_summary import ModelSummary

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
dm = TrackingDataModule(
    train=dict(
        dirs=[
            "/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part_1/"
        ],
    ),
    val=dict(
        dirs=[
            "/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part_9/"
        ],
        stop=5,
    ),
)

In [7]:
# Add path to model exchange
model_exchange_path = Path("/home/kl5675/Documents/23/git_sync/model-exchange/models")
assert model_exchange_path.exists(), model_exchange_path

In [8]:
lmodel = TCModule.load_from_checkpoint(model_exchange_path / "tc-04b2e3ce.ckpt")

[20:53:08] DEBUG: Getting class PreTrainedECGraphTCN from module gnn_tracking.models.track_condensation_networks
[20:53:08] DEBUG: Getting class ECForGraphTCN from module gnn_tracking.models.edge_classifier
/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:196: UserWarning: Attribute 'ec' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['ec'])`.
  rank_zero_warn(
/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:196: UserWarning: Attribute 'hc_in' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['hc_in'])`.
  rank_zero_warn(
[20:53:08] DEBUG: Getting class MLGraphConstruction from module gnn_tracking.models.graph_construction
[20:53:08] DEBUG: Getting class GraphConstructionFCN

In [14]:
ModelSummary(lmodel)

  | Name            | Type                 | Params
---------------------------------------------------------
0 | model           | PreTrainedECGraphTCN | 1.8 M 
1 | preproc         | MLGraphConstruction  | 1.3 M 
2 | potential_loss  | PotentialLoss        | 0     
3 | background_loss | BackgroundLoss       | 0     
---------------------------------------------------------
1.8 M     Trainable params
1.3 M     Non-trainable params
3.2 M     Total params
12.636    Total estimated model params size (MB)

In [9]:
pprint(
    dict(lmodel.hparams),
)

{
│   'model': {
│   │   'class_path': 'gnn_tracking.models.track_condensation_networks.PreTrainedECGraphTCN',
│   │   'init_args': {
│   │   │   'node_indim': 22,
│   │   │   'edge_indim': 44,
│   │   │   'h_dim': 192,
│   │   │   'e_dim': 192,
│   │   │   'h_outdim': 12,
│   │   │   'hidden_dim': 192,
│   │   │   'L_hc': 3,
│   │   │   'alpha_hc': 0.5,
│   │   │   'feed_edge_weights': True,
│   │   │   'ec_threshold': 0.2,
│   │   │   'mask_orphan_nodes': False,
│   │   │   'use_ec_embeddings_for_hc': True,
│   │   │   'ec': {
│   │   │   │   'class_path': 'gnn_tracking.models.edge_classifier.ECForGraphTCN',
│   │   │   │   'init_args': {
│   │   │   │   │   'node_indim': 22,
│   │   │   │   │   'edge_indim': 44,
│   │   │   │   │   'interaction_node_dim': 128,
│   │   │   │   │   'interaction_edge_dim': 128,
│   │   │   │   │   'hidden_dim': 128,
│   │   │   │   │   'L_ec': 3,
│   │   │   │   │   'alpha': 0.35,
│   │   │   │   │   'residual_type': 'skip1',
│   │   │   │   │   'use_intermediate_edge_embeddings': False,
│   │   │   │   │   'use_node_embedding': True,
│   │   │   │   │   'residual_kwargs': None
│   │   │   │   }
│   │   │   }
│   │   }
│   },
│   'preproc': {
│   │   'class_path': 'gnn_tracking.models.graph_construction.MLGraphConstruction',
│   │   'init_args': {
│   │   │   'max_radius': 0.8,
│   │   │   'max_num_neighbors': 64,
│   │   │   'use_embedding_features': True,
│   │   │   'ratio_of_false': None,
│   │   │   'build_edge_features': True,
│   │   │   'ec_threshold': None,
│   │   │   'ml_freeze': True,
│   │   │   'ec_freeze': True,
│   │   │   'ml': {
│   │   │   │   'class_path': 'gnn_tracking.models.graph_construction.GraphConstructionFCNN',
│   │   │   │   'init_args': {'in_dim': 14, 'hidden_dim': 512, 'out_dim': 8, 'depth': 6, 'beta': 0.4}
│   │   │   },
│   │   │   'ec': None
│   │   }
│   },
│   'lw_repulsive': 0.743380428762342,
│   'lw_background': 0.0041,
│   'potential_loss': {
│   │   'class_path': 'gnn_tracking.metrics.losses.PotentialLoss',
│   │   'init_args': {'q_min': 0.01, 'radius_threshold': 1.0, 'attr_pt_thld': 0.9}
│   },
│   'background_loss': {'class_path': 'gnn_tracking.metrics.losses.BackgroundLoss', 'init_args': {'sb': 0.1}},
│   'cluster_scanner': {
│   │   'class_path': 'gnn_tracking.postprocessing.dbscanscanner.DBSCANHyperParamScanner',
│   │   'init_args': {
│   │   │   'eps_range': (1e-05, 1.0),
│   │   │   'min_samples_range': (1, 1),
│   │   │   'n_jobs': 6,
│   │   │   'guide': 'trk.double_majority_pt0.9',
│   │   │   'n_trials': 20
│   │   }
│   }
}

In [10]:
trainer = Trainer(accelerator="gpu")

/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3 ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [11]:
trainer.validate(lmodel, dm, ckpt_path="last")

Missing logger folder: /home/kl5675/Documents/23/git_sync/lightning_logs
/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:189: UserWarning: .validate(ckpt_path="last") is set, but there is no last checkpoint available. No checkpoint will be loaded.
  rank_zero_warn(
You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
[20:53:13] INFO: DataLoader will load 5 graphs (out of 1000 available).
[20:53:13] DEBUG: First graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part_9/data29000_s0.pt, last graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part

Validation: 0it [00:00, ?it/s]

[20:53:26 ClusterHP] DEBUG: Starting from params: {}
[20:53:26 ClusterHP] INFO: Starting hyperparameter scan for clustering
[20:53:30 ClusterHP] DEBUG: Evaluated {'eps': 0.14238698036843575, 'min_samples': 1}: 0.4486756606201169
[20:53:32 ClusterHP] DEBUG: Evaluated {'eps': 0.2389268795921418, 'min_samples': 1}: 0.7735869964952273
[20:53:33 ClusterHP] DEBUG: Evaluated {'eps': 0.042176424999791864, 'min_samples': 1}: 0.0015724999590698472
[20:53:33 ClusterHP] DEBUG: Evaluated {'eps': 0.3000836829333336, 'min_samples': 1}: 0.8558276380757052
[20:53:39 ClusterHP] DEBUG: Evaluated {'eps': 0.2451274265894671, 'min_samples': 1}: 0.7856849277395281
[20:53:44 ClusterHP] DEBUG: Evaluated {'eps': 0.5752388363321559, 'min_samples': 1}: 0.8238898935086467
[20:53:49 ClusterHP] DEBUG: Evaluated {'eps': 0.6642813124049683, 'min_samples': 1}: 0.7275165776873436
[20:53:55 ClusterHP] DEBUG: Evaluated {'eps': 0.5224466198293832, 'min_samples': 1}: 0.8566001953945476
[20:53:56 ClusterHP] DEBUG: Evaluated 

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃            Validate metric             ┃              DataLoader 0              ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│             adjusted_rand              │          0.032201770693063736          │
│           adjusted_rand_std            │          0.003674338571727276          │
│         adjusted_rand_std_std          │                  nan                   │
│               attractive               │         1.3140039300196804e-05         │
│             attractive_std             │          2.98105561569173e-07          │
│          attractive_weighted           │         1.3140039300196804e-05         │
│        attractive_weighted_std         │          2.98105561569173e-07          │
│               background               │                  1.0                   │
│             background_std             │         1.332800358966324e-08          │
│          background_weighted           │          0.00409999955445528           │
│        background_weighted_std         │         1.0412502804424406e-10         │
│            best_dbscan_eps             │          0.41003432869911194           │
│          best_dbscan_eps_std           │                  nan                   │
│        best_dbscan_min_samples         │                  1.0                   │
│      best_dbscan_min_samples_std       │                  nan                   │
│              completeness              │           0.8836681842803955           │
│            completeness_std            │          0.003338507143780589          │
│          completeness_std_std          │                  nan                   │
│            fowlkes_mallows             │          0.03741481527686119           │
│          fowlkes_mallows_std           │          0.00333808409050107           │
│        fowlkes_mallows_std_std         │                  nan                   │
│              homogeneity               │           0.9329334497451782           │
│            homogeneity_std             │          0.006387058179825544          │
│          homogeneity_std_std           │                  nan                   │
│               repulsive                │         2.991704059240874e-05          │
│             repulsive_std              │         1.1564802662178408e-06         │
│           repulsive_weighted           │         2.2239744794205762e-05         │
│         repulsive_weighted_std         │         8.597047553848824e-07          │
│                 total                  │          0.004135379567742348          │
│               total_std                │         1.1014168421752402e-06         │
│          trk.double_majority           │           0.5158279538154602           │
│       trk.double_majority_pt0.5        │           0.777424156665802            │
│     trk.double_majority_pt0.5_std      │          0.011978638358414173          │
│   trk.double_majority_pt0.5_std_std    │                  nan                   │
│       trk.double_majority_pt0.9        │           0.8915719985961914           │
│     trk.double_majority_pt0.9_std      │          0.01321803592145443           │
│   trk.double_majority_pt0.9_std_std    │                  nan                   │
│       trk.double_majority_pt1.5        │           0.8886311054229736           │
│     trk.double_majority_pt1.5_std      │          0.025097912177443504          │
│   trk.double_majority_pt1.5_std_std    │                  nan                   │
│        trk.double_majority_std         │          0.009354520589113235          │
│      trk.double_majority_std_std       │                  nan                   │
│        trk.fake_double_majority        │          0.10832616686820984           │
│     trk.fake_double_majority_pt0.5     │          0.05339313670992851           │
│   trk.fake_double_majority_pt0.5_std   │          

[{'attractive': 1.3140039300196804e-05,
  'repulsive': 2.991704059240874e-05,
  'background': 1.0,
  'attractive_weighted': 1.3140039300196804e-05,
  'repulsive_weighted': 2.2239744794205762e-05,
  'background_weighted': 0.00409999955445528,
  'total': 0.004135379567742348,
  'v_measure': 0.907615602016449,
  'homogeneity': 0.9329334497451782,
  'completeness': 0.8836681842803955,
  'trk.n_particles': 7648.39990234375,
  'trk.n_cleaned_clusters': 4771.39990234375,
  'trk.perfect': 0.27800264954566956,
  'trk.double_majority': 0.5158279538154602,
  'trk.lhc': 0.9040055871009827,
  'trk.fake_perfect': 0.3461515009403229,
  'trk.fake_double_majority': 0.10832616686820984,
  'trk.fake_lhc': 0.09599439799785614,
  'trk.n_particles_pt0.5': 3414.800048828125,
  'trk.n_cleaned_clusters_pt0.5': 2835.39990234375,
  'trk.perfect_pt0.5': 0.48212671279907227,
  'trk.double_majority_pt0.5': 0.777424156665802,
  'trk.lhc_pt0.5': 0.929047167301178,
  'trk.fake_perfect_pt0.5': 0.34869056940078735,
  't